In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import matplotlib.image as img
import os
import glob
import shutil
import random
import pandas as pd
from PIL import Image
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
# Parameters
rim = 200 #resizing the image
val = 0.1 #10% of the training set data for the validation set
hu = 750 #hidden units
rel = 1

In [ ]:
# Importing images from the training set (and the test set)

def importing(path):
    spl = []
    for filename in glob.glob(path):
        im = Image.open(filename,'r')
        im = im.resize((rim,rim))#resizing the image
        im = np.array(im)
        IM = im.ravel() #contiguous flattened array (1D)
        spl.append(IM)
    return spl

# Path of the cancer cell images (Acute Lymphoblastic Leukemia) from the training set - all
path1 = '//kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0/all/*.bmp'
path2 = '//kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_1/all/*.bmp'
path3 = '//kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_2/all/*.bmp'

#import of cancer cells
a1 = importing(path1) 
print("a1: size  = ", len(a1))
a2 = importing(path2)
print("a2: size  = ", len(a2))
a3 = importing(path3)
print("a3: size  = ", len(a3))

train_a = []

for i in a1:
    train_a.append(i)#adding a cancer cell image to the relevant list
for i in a2:
    train_a.append(i)
for i in a3:
    train_a.append(i)

# Path of non-cancerous cell images from the training set - hem
path4 = '//kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0/hem/*.bmp'
path5 = '//kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_1/hem/*.bmp'
path6 = '//kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_2/hem/*.bmp'

#import of non-cancerous cells
h1 = importing(path4)
print("h1: size  = ", len(h1))
h2 = importing(path5)
print("h2: size  = ", len(h2))
h3 = importing(path6)
print("h3: size  = ", len(h3))

train_h = []

for i in h1:
    train_h.append(i) #adding a non-cancerous cell image to the relevant list
for i in h2:
    train_h.append(i)
for i in h3:
    train_h.append(i)

#testing_data
path7 = '../input/leukemia-classification/C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/*.bmp'


#import of test set images
t = importing(path7)
print("t: size  = ", len(t))

test_t = []

for i in t:
    test_t.append(i) #add a test set cell image to the relevant list


In [ ]:
#example of an image from the training set representing a cancerous image (Acute Lymphoblastic Leukemia)
image = img.imread('../input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0/all/UID_11_10_1_all.bmp')
plt.imshow(image)
plt.show()


#example of a training set image representing a non-cancerous image
image = img.imread('../input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0/hem/UID_H11_10_1_hem.bmp')
plt.imshow(image)
plt.show()

In [ ]:
dataframe_train_a = pd.DataFrame({'image':train_a, 'label': 'hem'}) #images of non-cancerous cells are assigned the label hem
dataframe_train_h = pd.DataFrame({'image':train_h, 'label': 'all'}) #images of cancer cells are labelled all (Acute Lymphoblastic Leukemia)

data_train = dataframe_train_a.append(dataframe_train_h,ignore_index=True) #the hem data is merged with the all data

In [ ]:
trainX = []
for i in data_train.image:
    trainX.append(i) #we assign the image to trainX
    
traindX = np.array(trainX)

In [ ]:
trainY = np.array(data_train.label) #assign the label to trainY

traindY = []
for i in range(len(trainY)):
    if(trainY[i]=="hem"):
        traindY.append(0) #hem is given the value 0
    else:
        traindY.append(1) #we give the value 1 to all
print(len(traindY))

classe = 2

trainY = np.zeros([len(traindY), classe])
for i in range(len(traindY)):
        trainY[i][traindY[i]] = 1

In [ ]:
# Division of the training set into validation (10%) and training set (90%)
trainX, validX, trainY, validY = train_test_split(traindX, trainY, test_size=val)
print('Training size: {training}, Validation size: {validation}'.format(training=len(trainX), validation=len(validX)))

In [ ]:
# Matrix transformation
trainX=np.matrix(trainX)
trainY=np.matrix(trainY)

validX=np.matrix(validX)
validY=np.matrix(validY)

test_t=np.matrix(test_t)

In [ ]:
# Extreme Learning Machine model
inputL = trainX.shape[1] #input length
HiddenUnits = hu #Hidden units
Wi = np.random.normal(size=[inputL, HiddenUnits]) #Gaussian random noise
Wi = np.matrix(Wi) #Hidden layer input weight matrix

print('Input weight shape: {shape}'.format(shape=Wi.shape))

In [ ]:
def input_hidden(x):
    print("x shape:",x.shape)
    print("Weights shape:",Wi.shape)
    
    m = np.dot(x, Wi) #product (x.Wi)
    m = np.maximum(m, rel, m) 
    return m

In [ ]:
X = input_hidden(trainX) 
X_tr = np.transpose(X)
ww = np.dot(X_tr, X) 
c = np.identity(len(np.array(ww)))
c = np.dot(c,3)
ww = ww + c
wo_1 = np.linalg.inv(ww) 

wo_2 = np.dot(X_tr, trainY) 

Wo = np.dot(wo_1, wo_2) #Final formula to be multiplied by x to get the prediction
print('Output weights shape: {shape}'.format(shape=Wo.shape))

In [ ]:
def pred(x):
    x = input_hidden(x)
    y = np.dot(x, Wo) #(x.Wo)
    return y

In [ ]:
# Prediction
accuracy_ =  []
specificty_ = []
sensibility = []

#confusion matrix, 1 = all = positive and 0 = hem = negative
TP = 0 #true positive
TN = 0 #true negative
FP = 0 #false positive
FN = 0 #false negative

y = pred(validX)
correct = 0
total = y.shape[0]
for i in range(total):
    prediction = np.argmax(y[i])
    valid = np.argmax(validY[i])
    correct = correct + (1 if prediction == valid else 0)
    TP = TP + (1 if prediction == valid == 1 else 0)
    TN = TN + (1 if prediction == valid == 0 else 0)
    FP = FP + (1 if prediction==1 and valid == 0 else 0)
    FN = FN + (1 if prediction==0 and valid == 1 else 0)

conf_matrix = [(TP, FP), (FN, TN)]
print('TP ' ,TP, '  ', 'FP ',FP)
print('FN ',FN, '  ', 'TN ',TN)

print('ACCURACY : ',(TP+TN)/(TP+TN+FP+FN))
accuracy_.append((TP+TN)/(TP+TN+FP+FN)) 

print('SENSIVITY : ', (TP)/(TP+FN))
sensibility.append((TP)/(TP+FN))

print('SPECIFICITY : ', (TN)/(TN+FP))
specificty_.append((TN)/(TN+FP))

In [ ]:
#testing_data prediction 
y=pred(test_t)
for i in range(total):
    prediction = np.argmax(y[i])
    if(prediction==1):
        prediction = "all"
    else:
        prediction = "hem"
    
    print(prediction)